In [1]:
using StatisticalRethinking, Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

352×2 Array{Float64,2}:
 1.0  47.8256
 1.0  36.4858
 1.0  31.8648
 1.0  53.0419
 1.0  41.2769
 1.0  62.9926
 1.0  38.2435
 1.0  55.48  
 1.0  34.8699
 1.0  54.4877
 ⋮           
 1.0  47.8823
 1.0  39.4058
 1.0  41.0501
 1.0  40.8233
 1.0  47.0318
 1.0  34.2462
 1.0  52.1631
 1.0  54.0625
 1.0  52.5316

Model Specification

In [3]:
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
NaN


Initial Values

In [4]:
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

3-element Array{Dict{Symbol,Any},1}:
 Dict(:beta=>[439.499, 13.0711],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>1.12646) 
 Dict(:beta=>[124.401, -2.44052],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>49.6957)
 Dict(:beta=>[273.268, 4.61792],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>36.6276) 

Tuning Parameters

In [5]:
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

0.1

Define sampling scheme

In [6]:
scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
NaN


MCMC Simulation

In [7]:
sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:17:31 of 0:17:32 remaining]
Chain 1:  10% [0:00:49 of 0:00:55 remaining]
Chain 1:  20% [0:00:58 of 0:01:12 remaining]
Chain 1:  30% [0:00:55 of 0:01:19 remaining]
Chain 1:  40% [0:00:47 of 0:01:19 remaining]
Chain 1:  50% [0:00:39 of 0:01:18 remaining]
Chain 1:  60% [0:00:30 of 0:01:16 remaining]
Chain 1:  70% [0:00:23 of 0:01:17 remaining]
Chain 1:  80% [0:00:15 of 0:01:17 remaining]
Chain 1:  90% [0:00:08 of 0:01:17 remaining]
Chain 1: 100% [0:00:00 of 0:01:17 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:01:00 of 0:01:07 remaining]
Chain 2:  20% [0:00:55 of 0:01:09 remaining]
Chain 2:  30% [0:00:49 of 0:01:10 remaining]
Chain 2:  40% [0:00:43 of 0:01:12 remaining]
Chain 2:  50% [0:00:36 of 0:01:13 remaining]
Chain 2:  60% [0:00:29 of 0:01:13 remaining]
Chain 2:  70% [0:00:22 of 0:01:14 remaining]
Chain 2:  80% [0:00:15 of 0:01:14 remaining]
Chain 2:  90% [0:00:07 of 0:01:15 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[24.2192 112.835 0.932352; 22.9597 112.835 0.932352; … ; 25.5319 114.865 0.885688; 26.2172 114.872 0.881323]

[25.8295 115.366 0.860628; 27.1481 115.367 0.891976; … ; 25.3696 113.028 0.925005; 25.1729 114.303 0.898536]

[27.1773 118.671 0.792342; 26.8226 118.015 0.811485; … ; 29.1453 112.35 0.948926; 29.6217 112.346 0.929553]

Show draws summary

In [8]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS   
     s2  26.1774380 2.026125536 0.01233060734 0.0208816097 9000.0000
beta[1] 113.9011788 1.925965426 0.01172105232 0.0390243979 2435.7042
beta[2]   0.9045543 0.042366338 0.00025783332 0.0008457032 2509.6083

Quantiles:
            2.5%        25.0%       50.0%        75.0%       97.5%   
     s2  22.4979817  24.78791273  26.0830224  27.47015216  30.4796982
beta[1] 110.1210475 112.63049891 113.9187543 115.16560785 117.8058074
beta[2]   0.8200496   0.87673887   0.9042193   0.93247695   0.9874822



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*